<a href="https://colab.research.google.com/github/aliakbarbadri/natural-language-inference/blob/master/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU, TimeDistributed
import unicodedata
import re
import numpy as np
import os
import io
import time
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

# Load and Preprocess the data

In [3]:
source = pd.read_pickle(r'https://github.com/aliakbarbadri/natural-language-inference/blob/master/hypotheses_train.pickle?raw=true')
target = pd.read_pickle(r'https://github.com/aliakbarbadri/natural-language-inference/blob/master/premises_train.pickle?raw=true')

In [5]:
print(source[0])
print(target[0])

('A person is outdoors , on a horse .',
 'A person on a horse jumps over a broken down airplane .')

In [13]:
lines = pd.DataFrame(
    {'source': source,
     'target': target
     })

In [14]:
lines.shape

(183416, 2)

In [15]:
# Lowercase all characters
lines.source=lines.source.apply(lambda x: x.lower())
lines.target=lines.target.apply(lambda x: x.lower())

# Remove quotes
lines.source=lines.source.apply(lambda x: re.sub("'", '', x))
lines.target=lines.target.apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.source=lines.source.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.target=lines.target.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.source=lines.source.apply(lambda x: x.translate(remove_digits))
lines.target=lines.target.apply(lambda x: x.translate(remove_digits))

# Remove extra spaces
lines.source=lines.source.apply(lambda x: x.strip())
lines.target=lines.target.apply(lambda x: x.strip())
lines.source=lines.source.apply(lambda x: re.sub(" +", " ", x))
lines.target=lines.target.apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines.target = lines.target.apply(lambda x : 'START_ '+ x + ' _END')

lines.sample(10)

,source,target
67803,child is sleeping on furniture,START_ little boy is sleeping on a brown sofa ...
650,some humans in a truck,START_ people in a truckfull of sacks in a fie...
95814,the man in the gray shirt is emotional,START_ a man in a gray tshirt and jeans smiles...
36180,a man finds a fish with half of its body missing,START_ a man is staring down at half of a fish...
80393,some people are playing jenga and some people ...,START_ a family sits around at christmas time ...
168113,cheerleaders are performing a stunt,START_ cheerleaders at a sporting event toss a...
65541,a daughter receives a trophy,START_ mother and daughter stand together happ...
161909,a lady in a sweater leans over an older person,START_ a woman wearing a pullover with an amer...
38464,a little dog is biting a stick,START_ a smaller white dog is trying to get a ...
51543,two ladies and a teenager show a puppy to a girl,START_ two ladies and a teenage girl all dress...
